In [1]:
import os

path = '../data/signac'
os.listdir(path)

['.empty',
 'atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5',
 'atac_v1_pbmc_10k_fragments.tsv.gz',
 'atac_v1_pbmc_10k_fragments.tsv.gz.tbi',
 'atac_v1_pbmc_10k_singlecell.csv',
 'hahah.loom',
 'pbmc_10k_v3.rds']

## First thing first, how to handle rds, or, to be more specific, Seurat object, in python

### Solutions I tried:

#### Python Packages

##### pyreadr

LibrdataError: The file contains an unrecognized object

Reason: from homepage (https://github.com/ofajardo/pyreadr#basic-usage--reading-files)

"It can read mainly R data frames. Support for other kind of R objects is limited. R lists and R S4 objects (such as those from Bioconductor) are not supported."

In [2]:
!pip install pyreadr

import pyreadr

df = pyreadr.read_r(f'{path}/pbmc_10k_v3.rds')

LibrdataError: The file contains an unrecognized object

##### rpy2 (Seurat required)

In [12]:
!pip install rpy2
!conda install -c bioconda r-seurat --quiet --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - r-seurat


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    certifi-2020.12.5          |   py38h578d9bd_0         143 KB  conda-forge
    colormath-3.0.0            |             py_2          34 KB  conda-forge
    r-abind-1.4_5              | r40h6115d3f_1003          76 KB  conda-forge
    r-bh-1.72.0_3              |    r40h6115d3f_1        10.4 MB  conda-forge
    r-catools-1.18.0           |    r40h0357c0b_1         222 KB  conda-forge
    r-cluster-2.1.0            |    r40h31ca83e_4         553 KB  conda-forge
    r-cowplot-1.1.0            |    r40h6115d3f_0         1.3 MB  conda-forge
    r-deldir-0.2_3             |    r40h13aaf81_0         261 KB  conda-forge
    r-dqrng-0.2

Work fine with normal R dataframe...

In [8]:
import rpy2.robjects as ro

r_df = ro.DataFrame({'int_values': ro.IntVector([1,2,3]),
                     'str_values': ro.StrVector(['abc', 'def', 'ghi'])})

r_df

int_values,str_values
...,...


In [9]:
with ro.conversion.localconverter(ro.default_converter + ro.pandas2ri.converter):
    pd_from_r_df = ro.conversion.rpy2py(r_df)
    
pd_from_r_df

AttributeError: module 'rpy2.robjects' has no attribute 'pandas2ri'

weired import behavior

In [10]:
from rpy2.robjects import pandas2ri

with ro.conversion.localconverter(ro.default_converter + ro.pandas2ri.converter):
    pd_from_r_df = ro.conversion.rpy2py(r_df)
    
pd_from_r_df

,int_values,str_values
1,1,abc
2,2,def
3,3,ghi


but not with Seurat object

In [11]:
r = ro.r

model_rds_path = f'{path}/pbmc_10k_v3.rds'
model = r.readRDS(model_rds_path)

with ro.conversion.localconverter(ro.default_converter + ro.pandas2ri.converter):
    pd_from_r_df = ro.conversion.rpy2py(model)

R[write to console]: Warning:
R[write to console]:  namespace ‘Seurat’ is not available and has been replaced
by .GlobalEnv when processing object ‘<unknown>’

R[write to console]: Loading required package: Seurat

R[write to console]: Error in .requirePackage(package) : 
  unable to find required package ‘Seurat’

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
R[write to console]: 
 
R[write to console]:  there is no package called ‘Seurat’



RRuntimeError: Error in .requirePackage(package) : 
  unable to find required package ‘Seurat’


##### feather

SSL is required.

In [10]:
!pip install feather

    ERROR: Command errored out with exit status 1:
     command: /opt/conda/bin/python3.8 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-l9b8_0zj/feather_529f10d460f44c9888005e95b3042343/setup.py'"'"'; __file__='"'"'/tmp/pip-install-l9b8_0zj/feather_529f10d460f44c9888005e95b3042343/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-6m18tqqc
         cwd: /tmp/pip-install-l9b8_0zj/feather_529f10d460f44c9888005e95b3042343/
    Complete output (31 lines):
    Traceback (most recent call last):
      File "/tmp/pip-install-l9b8_0zj/feather_529f10d460f44c9888005e95b3042343/distribute_setup.py", line 143, in use_setuptools
        raise ImportError
    ImportError
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "<string

In [12]:
import gzip

def less(file, bin=True):
    if bin is True:
        with gzip.open(f'{path}/{file}', 'rb') as f:
            for lineno, i in enumerate(f):
                if lineno < 100:
                    print(i)
                else:
                    break
    else:
        with open(f'{path}/{file}', 'r') as f:
            for lineno, i in enumerate(f):
                if lineno < 100:
                    print(i)
                else:
                    break

In [9]:
less('atac_v1_pbmc_10k_fragments.tsv.gz.tbi')

b'TBI\x01\x18\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00#\x00\x00\x00\x00\x00\x00\x00\x85\x00\x00\x00chr1\x00chr2\x00chr3\x00chr4\x00chr5\x00chr6\x00chr7\x00chr8\x00chr9\x00chr10\x00chr11\x00chr12\x00chr13\x00chr14\x00chr15\x00chr16\x00chr17\x00chr18\x00chr19\x00chr20\x00chr21\x00chr22\x00chrX\x00chrY\x00\x89\x06\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\xa0\x93\x9e\x90\xef\x03\x00\x00\xdc\x95\x9e\x90\xef\x03\x00\x00\x15\xd8\t\xe8\x81\x08\x00\x00\x1d\xdd\t\xe8\x81\x08\x00\x00\x01\x00\x00\x00\x06\x00\x00\x001\xd3yE\x8c\x00\x00\x00\xcd\xd6yE\x8c\x00\x00\x00{_\xed\x84\r\x01\x00\x00\xb7a\xed\x84\r\x01\x00\x00_\xbb\xde\xc8\x8f\x01\x00\x00\x93\xbc\xde\xc8\x8f\x01\x00\x00\xe2\x0f\xbdRI\x02\x00\x00~\x1e\xbdRI\x02\x00\x00\xc4\x84\x93V\xca\x02\x00\x00\xd4\x86\x93V\xca\x02\x00\x009K\xda\xf0B\x03\x00\x00uM\xda\xf0B\x03\x00\x00\x02\x00\x00\x00\x06\x00\x00\x00\xbdx\x83\x87)\x04\x00\x00Iz\x83\x87)\x04\x00\x00\x1d\xe9\x9b\xc0\\\x04\x00\x00I\xe9\x9b\xc0\\\x04\x00\x00ZTv\x

In [10]:
less('atac_v1_pbmc_10k_fragments.tsv.gz')

b'chr1\t10066\t10279\tTTAGCTTAGGAGAACA-1\t2\n'
b'chr1\t10072\t10279\tTTAGCTTAGGAGAACA-1\t2\n'
b'chr1\t10079\t10316\tATATTCCTCTTGTACT-1\t2\n'
b'chr1\t10084\t10340\tCGTACAAGTTACCCAA-1\t1\n'
b'chr1\t10085\t10271\tTGTGACAGTACAACGG-1\t1\n'
b'chr1\t10085\t10339\tCATGCCTTCTCTGACC-1\t1\n'
b'chr1\t10089\t10502\tATCGAGTAGGTTCGAG-1\t1\n'
b'chr1\t10091\t10334\tGCGGTGTCACACTAGT-1\t1\n'
b'chr1\t10091\t10681\tCTCTCAGGTCCCTTTG-1\t1\n'
b'chr1\t10096\t10278\tTTCGGTCTCACGTGTA-1\t1\n'
b'chr1\t10096\t10339\tCTTTGCGAGGTACACG-1\t1\n'
b'chr1\t10097\t10321\tTTGCGGGCAAGCGGTA-1\t1\n'
b'chr1\t10097\t10334\tAACCAACGTGTCTACA-1\t2\n'
b'chr1\t10101\t10483\tCTGTTCGGTGCGTTTA-1\t2\n'
b'chr1\t10132\t10279\tTGGCCTTAGAGCACTG-1\t2\n'
b'chr1\t10136\t10483\tCTGTTCGGTGCGTTTA-1\t1\n'
b'chr1\t10143\t10327\tTGTACAGGTGGCATAG-1\t1\n'
b'chr1\t10145\t10473\tGGGTTATGTCTCTGGG-1\t1\n'
b'chr1\t10151\t10209\tCGCGCAACAAGATTAG-1\t2\n'
b'chr1\t10151\t10565\tGTAATCGAGTTGTCCA-1\t1\n'
b'chr1\t10157\t10210\tTTAGCTTAGTGATTAG-1\t2\n'
b'chr1\t10157

In [13]:
less('atac_v1_pbmc_10k_singlecell.csv', bin=False)

barcode,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,cell_id,is__cell_barcode,TSS_fragments,DNase_sensitive_region_fragments,enhancer_region_fragments,promoter_region_fragments,on_target_fragments,blacklist_region_fragments,peak_region_fragments

NO_BARCODE,8335327,3922818,71559,746476,634014,22972,2937488,None,0,0,0,0,0,0,0,0

AAACGAAAGAAACGCC-1,3,1,0,1,0,0,1,None,0,1,1,1,0,1,0,1

AAACGAAAGAAAGCAG-1,14,1,0,4,2,0,7,None,0,0,2,0,0,2,0,0

AAACGAAAGAAAGGGT-1,7,1,0,1,0,0,5,None,0,2,2,0,1,2,0,1

AAACGAAAGAAATACC-1,9880,5380,79,106,1120,6,3189,None,0,386,1623,297,107,1758,16,262

AAACGAAAGAAATCTG-1,2,0,0,0,0,0,2,None,0,1,2,1,1,2,0,2

AAACGAAAGAAATGGG-1,4,2,0,1,0,0,1,None,0,0,1,1,0,1,0,1

AAACGAAAGAAATTCG-1,2,0,0,0,1,0,1,None,0,0,0,0,0,0,0,0

AAACGAAAGAACCATA-1,12,3,0,8,0,0,1,None,0,0,1,0,0,1,0,0

AAACGAAAGAACCCGA-1,3,1,0,2,0,0,0,None,0,0,0,0,0,0,0,0

AAACGAAAGAACGACC-1,2,0,0,0,0,0,2,None,0,2,2,1,1,2,0,1

AAACGAAAGAACGCCA-1,7,0,0,1,0,0,6,None,0,0,4,0,0,4,0,0

AAACGAA

In [25]:
!conda install -c bioconda r-seurat

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - r-seurat


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    certifi-2020.12.5          |   py38h578d9bd_0         143 KB  conda-forge
    colormath-3.0.0            |             py_2          34 KB  conda-forge
    r-abind-1.4_5              | r40h6115d3f_1003          76 KB  conda-forge
    r-bh-1.72.0_3              |    r40h6115d3f_1        10.4 MB  conda-forge
    r-catools-1.18.0           |    r40h0357c0b_1         222 KB  conda-forge
    r-cluster-2.1.0            |    r40h31ca83e_4         553 KB  conda-forge
    r-cowplot-1.1.0            |    r40h6115d3f_0         1.3 MB  conda-forge
    r-deldir-0.2_3             |    r40h13aaf81_0         261 KB  conda-forge
    r-dqrng-0.2.1            

In [ ]:
!pip install rpy2

In [31]:
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri, pandas2ri
from rpy2.robjects.packages import importr

In [28]:
r = ro.r
numpy2ri.activate()

model_rds_path = f'{path}/pbmc_10k_v3.rds'
model = r.readRDS(model_rds_path)

In [30]:
from rpy2.robjects.conversion import localconverter

In [33]:
with localconverter(ro.default_converter + pandas2ri.converter):
    pd_from_r_df = ro.conversion.rpy2py(model)

In [34]:
pd_from_r_df

R object with classes: ('Seurat',) mapped to:

In [35]:
r_df = ro.DataFrame({'int_values': ro.IntVector([1,2,3]),
                     'str_values': ro.StrVector(['abc', 'def', 'ghi'])})

r_df

int_values,str_values
...,...


In [36]:
with localconverter(ro.default_converter + pandas2ri.converter):
    pd_from_r_df = ro.conversion.rpy2py(r_df)

pd_from_r_df

,int_values,str_values
1,1,abc
2,2,def
3,3,ghi


In [37]:
model

R object with classes: ('Seurat',) mapped to:

In [1]:
import h5py

In [44]:
def check_h5(filename):
    with h5py.File(f'{path}/{filename}') as f:
        print(type(f))

        for i in f.keys():
            if isinstance(f[i], h5py.Dataset):
                print('Dataset ---- ', end='')
            else:
                print('Group ---- ', end='')
            print(i, f[i], f[i].name)

In [45]:
check_h5('atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5')

<class 'h5py._hl.files.File'>
Group ---- matrix <HDF5 group "/matrix" (6 members)> /matrix


In [46]:
check_h5('hahah.loom')

<class 'h5py._hl.files.File'>
Group ---- col_attrs <HDF5 group "/col_attrs" (14 members)> /col_attrs
Group ---- col_graphs <HDF5 group "/col_graphs" (2 members)> /col_graphs
Group ---- layers <HDF5 group "/layers" (2 members)> /layers
Dataset ---- matrix <HDF5 dataset "matrix": shape (19089, 9432), type "<f8"> /matrix
Group ---- row_attrs <HDF5 group "/row_attrs" (7 members)> /row_attrs
Group ---- row_graphs <HDF5 group "/row_graphs" (0 members)> /row_graphs


In [51]:
filename = 'hahah.loom'
with h5py.File(f'{path}/{filename}') as f:
        print(type(f))

        for i in f.keys():
            if isinstance(f[i], h5py.Dataset):
                print('Dataset ---- ', end='')
            else:
                print('Group ---- ', end='')
            print(i, f[i], f[i].name)
        print()
        group = f['layers']
        for i in group.keys():
            if isinstance(group[i], h5py.Dataset):
                print('Dataset ---- ', end='')
            else:
                print('Group ---- ', end='')
            print(i, group[i], group[i].name)
        print()
        group = f['col_attrs']
        for i in group.keys():
            if isinstance(group[i], h5py.Dataset):
                print('Dataset ---- ', end='')
            else:
                print('Group ---- ', end='')
            print(i, group[i], group[i].name)

<class 'h5py._hl.files.File'>
Group ---- col_attrs <HDF5 group "/col_attrs" (14 members)> /col_attrs
Group ---- col_graphs <HDF5 group "/col_graphs" (2 members)> /col_graphs
Group ---- layers <HDF5 group "/layers" (2 members)> /layers
Dataset ---- matrix <HDF5 dataset "matrix": shape (19089, 9432), type "<f8"> /matrix
Group ---- row_attrs <HDF5 group "/row_attrs" (7 members)> /row_attrs
Group ---- row_graphs <HDF5 group "/row_graphs" (0 members)> /row_graphs

Dataset ---- norm_data <HDF5 dataset "norm_data": shape (19089, 9432), type "<f8"> /layers/norm_data
Dataset ---- scale_data <HDF5 dataset "scale_data": shape (19089, 9432), type "<f8"> /layers/scale_data

Dataset ---- CellID <HDF5 dataset "CellID": shape (9432,), type "|S22"> /col_attrs/CellID
Dataset ---- ClusterID <HDF5 dataset "ClusterID": shape (9432,), type "<i4"> /col_attrs/ClusterID
Dataset ---- ClusterName <HDF5 dataset "ClusterName": shape (9432,), type "|S22"> /col_attrs/ClusterName
Dataset ---- RNA_snn_res_0_4 <HDF5 da

In [35]:
with h5py.File(f'{path}/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5') as f:
    shape = f['matrix'].get('shape')
    print(shape[:])

[89796  8728]


<Closed HDF5 dataset>

In [13]:
print(1)

1
